In [25]:
import nltk
import pandas as pd
import pymorphy2

In [53]:
def find_all_definitions(df, idx):
    stop_simbols = ['(', ')', '[', ']', '{', '}', ':', ';', '?', '!']
    abbreviation = df['token'][idx].lower()
    abbr_len = len(abbreviation)
    max_len = min(abbr_len+5, 2*abbr_len)
    all_definitions = {}
    begin = {}
    end = {}
    L = max(0, idx-10*max_len)
    R = min(len(df), idx+10*max_len+1)
    for i in range (L, R):
        if i==idx:
            continue
        if abbreviation[0]!=(df['token'][i].lower())[0]:
            continue
        for k in range(i, min(i+max_len+1, len(df))):
            definition = []
            for j in range(i, k):
                if j==idx or df['token'][j] in stop_simbols or df['is_abbreviation'][j]==1:
                    break
                definition.append(df['token'][j])
            if len(definition)==0:
                continue
            lost_letters = 0
            lost_simbols = 0
            for ch in abbreviation:
                for w in definition:
                    if ch in w:
                        break
                else:
                    if ch.isalpha():
                        lost_letters += 1
                    else:
                        lost_simbols += 1
            if (len(abbreviation)<=2 and lost_letters==0 or 
                len(abbreviation)>2 and lost_letters<=0 and
                lost_simbols<=1):
                distation = min(abs(i-idx), abs(j-idx))
                d = ' '.join(definition)
                try:
                    if all_definitions[d]>distation:
                        all_definitions[d] = distation
                        begin[d] = i
                        end[d] = k
                except:
                    all_definitions[d] = distation
                    begin[d] = i
                    end[d] = k
    return all_definitions, begin, end

In [61]:
def find_abbreviation(df):
    abbreviations = []
    definitions = []
    distance = []
    begins = []
    ends = []
    abbreviation_place = []
    for i in range(len(df)):
        if(df['is_abbreviation'][i]==1):
            definitions_of_this, begin, end = find_all_definitions(df, i)
            for d in definitions_of_this.keys():
                abbreviations.append(df['token'][i])
                abbreviation_place.append(i)
                definitions.append(d)
                distance.append(definitions_of_this[d])
                begins.append(begin[d])
                ends.append(end[d])
    new_df = pd.DataFrame({'abbreviation' : abbreviations, 
                           'definition' : definitions, 'distance' : distance,
                            'begin' : begins, 'end' : ends, 
                            'abbreviation_place' : abbreviation_place})
    return new_df

In [62]:
list_of_words = pd.read_excel("../dataset/ready_data/merges_data.xlsx")
pairs = find_abbreviation(list_of_words)
pairs.to_csv("../dataset/ready_data/potential_pairs.csv")

In [63]:
list_of_words = pd.read_excel("../dataset/ready_data/6_without_stopwords.xlsx")
pairs = find_abbreviation(list_of_words)
pairs.to_csv("../dataset/ready_data/6_potential_pairs.csv")